## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hovd,MN,48.0056,91.6419,66.04,30,100,7.23,overcast clouds
1,1,Cherlak,RU,54.1550,74.8055,57.36,80,66,9.80,broken clouds
2,2,Hobart,AU,-42.8794,147.3294,43.27,94,64,11.50,broken clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,51.80,58,75,11.50,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,72.54,66,13,4.85,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Harper,LR,4.3750,-7.7169,76.44,87,100,10.29,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,76.96,94,75,8.01,broken clouds
14,14,Tarauaca,BR,-8.1614,-70.7656,85.30,57,76,2.15,broken clouds
17,17,Cedar City,US,37.6775,-113.0619,88.39,6,0,15.95,clear sky
25,25,Hilo,US,19.7297,-155.0900,77.61,100,100,4.61,moderate rain
31,31,Albany,US,42.6001,-73.9662,76.73,72,91,9.98,overcast clouds
32,32,Fonte Boa,BR,-2.4959,-66.2662,81.41,81,38,1.77,scattered clouds
44,44,Axim,GH,4.8699,-2.2405,75.04,89,100,12.68,moderate rain
53,53,Puerto Leguizamo,CO,-0.1934,-74.7819,79.93,71,99,3.11,overcast clouds
54,54,Merauke,ID,-8.4667,140.3333,77.79,89,24,12.84,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count() 

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy() 
clean_df.dropna(inplace = True)  
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Harper,LR,76.44,overcast clouds,4.3750,-7.7169,
9,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
14,Tarauaca,BR,85.30,broken clouds,-8.1614,-70.7656,
17,Cedar City,US,88.39,clear sky,37.6775,-113.0619,
25,Hilo,US,77.61,moderate rain,19.7297,-155.0900,
31,Albany,US,76.73,overcast clouds,42.6001,-73.9662,
32,Fonte Boa,BR,81.41,scattered clouds,-2.4959,-66.2662,
44,Axim,GH,75.04,moderate rain,4.8699,-2.2405,
53,Puerto Leguizamo,CO,79.93,overcast clouds,-0.1934,-74.7819,
54,Merauke,ID,77.79,few clouds,-8.4667,140.3333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.dropna(inplace = True)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Harper,LR,76.44,overcast clouds,4.3750,-7.7169,Screensaver
9,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Tarauaca,BR,85.30,broken clouds,-8.1614,-70.7656,Flat Temporada
17,Cedar City,US,88.39,clear sky,37.6775,-113.0619,Hampton Inn Cedar City
25,Hilo,US,77.61,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))